<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/getScreenerData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import openpyxl

from google.colab import drive
drive.mount('/content/drive')

from google.colab import data_table

#mypf = pd.read_csv('/content/drive/My Drive/data/stocks/Try-Hyperlink.xlsx')


def _get_link_if_exists(cell) -> str | None:
    try:
        return cell.hyperlink.target
    except AttributeError:
        return None


def extract_hyperlinks_from_xlsx(
    file_name: str, sheet_name: str, columns_to_parse: list[str], row_header: int = 1
) -> pd.DataFrame:
    df = pd.read_excel(file_name, sheet_name)
    ws = openpyxl.load_workbook(file_name)[sheet_name]
    for column in columns_to_parse:
        row_offset = row_header + 1
        column_index = list(df.columns).index(column) + 1
        df[column] = [
            _get_link_if_exists(ws.cell(row=row_offset + i, column=column_index))
            for i in range(len(df[column]))
        ]
    return df

df_data = extract_hyperlinks_from_xlsx('/content/drive/My Drive/quantvesting/data/myScreenerDB.xlsx','data', ['Name'])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
df_data

,Name,CMP Rs.,ATH%,P/E,EPS 12M Rs.,CMP / BV,Mar Cap Rs.Cr.,ROCE %,ROE %,Sales growth %,Profit growth %,5Yrs PE,ROE/PE,ROE 5Yr %,Debt / Eq,NP 12M Rs.Cr.,CFO to EBITDA %,CapType
0,https://www.screener.in/company/RELIANCE/conso...,1510.60,6.51,26.66,61.41,2.33,2043876.70,9.69,8.40,8.08,12.85,26.21,0.32,8.45,0.43,97416.00,98.18,LC
1,https://www.screener.in/company/HDFCBANK/conso...,986.80,3.42,20.97,47.22,2.81,1517448.80,7.51,14.45,7.51,4.60,20.42,0.69,16.13,6.30,75078.88,44.84,LC
2,https://www.screener.in/company/BHARTIARTL/con...,2088.70,2.25,38.91,67.62,10.11,1253057.59,13.48,23.18,25.17,127.00,63.32,0.60,14.88,1.77,44683.00,106.91,LC
3,https://www.screener.in/company/TCS/consolidated/,3106.00,47.85,22.35,136.64,10.56,1123975.37,64.63,52.38,3.62,4.43,30.90,2.34,46.84,0.10,49687.00,68.57,LC
4,https://www.screener.in/company/ICICIBANK/cons...,1385.95,8.23,18.58,74.91,3.14,990378.27,7.87,17.89,10.50,12.52,19.64,0.96,16.79,5.93,56975.06,74.67,LC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,https://www.screener.in/company/JSLL/,737.70,15.22,59.58,12.39,33.58,9172.72,50.24,37.93,62.41,82.71,46.88,0.64,40.34,0.04,153.95,53.45,SC
596,https://www.screener.in/company/BIRLACORPN/con...,1186.90,51.82,17.34,64.65,1.28,9143.03,7.08,4.79,6.86,72.90,26.56,0.28,6.30,0.45,497.84,126.62,SC
597,https://www.screener.in/company/ORKLAINDIA/con...,664.85,14.31,35.28,186.65,NaN,9102.96,13.55,9.80,1.64,18.03,3.62,0.28,10.28,0.02,255.69,91.82,SC
598,https://www.screener.in/company/BORORENEW/cons...,647.50,28.70,NaN,-14.10,9.66,9088.58,-4.41,-8.90,7.63,78.48,93.94,NaN,NaN,0.23,-201.52,132.17,SC


In [3]:
df_data.columns

Index(['Name', 'CMP Rs.', 'ATH%', 'P/E', 'EPS 12M Rs.', 'CMP / BV',
       'Mar Cap Rs.Cr. ', 'ROCE %', 'ROE %', 'Sales growth %',
       'Profit growth %', '5Yrs PE', 'ROE/PE', 'ROE 5Yr %', 'Debt / Eq',
       'NP 12M Rs.Cr.', 'CFO to EBITDA %', 'CapType'],
      dtype='object')

In [4]:
df_data['Symbol'] = df_data['Name'].str.split('company/').str[1].str.split('/').str[0]
df_data['Latest']  = 1
df_data.columns = ['Name', 'CMP', 'ATH%', 'PE', 'EPS', 'PB', 'MCap', 'ROCE%', 'ROE%', 'Sales_Grwth%', 'Profit_Grwth%', 'MedPE', 'ROCE_5Yr%',
       'ROE_5Yr%', 'Debt2EqR', 'PAT_12M', 'CFO_2_EBITDA%', 'CapType', 'Symbol', 'Latest']

In [5]:
df_data

,Name,CMP,ATH%,PE,EPS,PB,MCap,ROCE%,ROE%,Sales_Grwth%,Profit_Grwth%,MedPE,ROCE_5Yr%,ROE_5Yr%,Debt2EqR,PAT_12M,CFO_2_EBITDA%,CapType,Symbol,Latest
0,https://www.screener.in/company/RELIANCE/conso...,1510.60,6.51,26.66,61.41,2.33,2043876.70,9.69,8.40,8.08,12.85,26.21,0.32,8.45,0.43,97416.00,98.18,LC,RELIANCE,1
1,https://www.screener.in/company/HDFCBANK/conso...,986.80,3.42,20.97,47.22,2.81,1517448.80,7.51,14.45,7.51,4.60,20.42,0.69,16.13,6.30,75078.88,44.84,LC,HDFCBANK,1
2,https://www.screener.in/company/BHARTIARTL/con...,2088.70,2.25,38.91,67.62,10.11,1253057.59,13.48,23.18,25.17,127.00,63.32,0.60,14.88,1.77,44683.00,106.91,LC,BHARTIARTL,1
3,https://www.screener.in/company/TCS/consolidated/,3106.00,47.85,22.35,136.64,10.56,1123975.37,64.63,52.38,3.62,4.43,30.90,2.34,46.84,0.10,49687.00,68.57,LC,TCS,1
4,https://www.screener.in/company/ICICIBANK/cons...,1385.95,8.23,18.58,74.91,3.14,990378.27,7.87,17.89,10.50,12.52,19.64,0.96,16.79,5.93,56975.06,74.67,LC,ICICIBANK,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,https://www.screener.in/company/JSLL/,737.70,15.22,59.58,12.39,33.58,9172.72,50.24,37.93,62.41,82.71,46.88,0.64,40.34,0.04,153.95,53.45,SC,JSLL,1
596,https://www.screener.in/company/BIRLACORPN/con...,1186.90,51.82,17.34,64.65,1.28,9143.03,7.08,4.79,6.86,72.90,26.56,0.28,6.30,0.45,497.84,126.62,SC,BIRLACORPN,1
597,https://www.screener.in/company/ORKLAINDIA/con...,664.85,14.31,35.28,186.65,NaN,9102.96,13.55,9.80,1.64,18.03,3.62,0.28,10.28,0.02,255.69,91.82,SC,ORKLAINDIA,1
598,https://www.screener.in/company/BORORENEW/cons...,647.50,28.70,NaN,-14.10,9.66,9088.58,-4.41,-8.90,7.63,78.48,93.94,NaN,NaN,0.23,-201.52,132.17,SC,BORORENEW,1


In [6]:
df_myscreener = pd.read_csv('/content/drive/My Drive/quantvesting/data/myScreenerDB.csv')
df_tmp = df_myscreener[~df_myscreener['Symbol'].isin(df_data['Symbol'])]
df_myscreener = pd.concat([df_data, df_tmp], ignore_index=True)

In [7]:
df_myscreener = df_myscreener.sort_values(by = 'MCap', ascending=False)
df_myscreener['CapType'] = 'SC'  # Default category
df_myscreener.iloc[:100, df_myscreener.columns.get_loc('CapType')] = 'LC'
df_myscreener.iloc[100:250, df_myscreener.columns.get_loc('CapType')] = 'MC'

In [8]:
df_myscreener.to_csv('/content/drive/My Drive/quantvesting/data/myScreenerDB.csv', index=False)

In [9]:
cols = ['Symbol','PE', 'EPS','MCap','ROCE%','ROE%','MedPE','ROCE_5Yr%','ROE_5Yr%','Debt2EqR','Sales_Grwth%', 'Profit_Grwth%','CFO_2_EBITDA%','PAT_12M','PB','CapType']
df = df_myscreener[cols]

In [10]:
df_tmp = df[(df['PE'] < df['MedPE']) & (df['Debt2EqR'] < .25) & (df['Sales_Grwth%'] > 15) & (df['Profit_Grwth%'] > 25)]
data_table.DataTable(df_tmp, include_index=False)

,Symbol,PE,EPS,MCap,ROCE%,ROE%,MedPE,ROCE_5Yr%,ROE_5Yr%,Debt2EqR,Sales_Grwth%,Profit_Grwth%,CFO_2_EBITDA%,PAT_12M,PB,CapType
66,AMBUJACEM,24.03,23.00,138051.07,10.50,8.73,32.71,0.36,9.76,0.02,17.61,94.08,26.09,7221.58,2.46,LC
113,POWERINDIA,133.47,162.51,95727.47,19.44,13.79,185.94,0.10,13.15,0.02,16.49,259.80,225.44,717.23,20.88,MC
128,NAUKRI,66.35,20.34,87414.31,3.00,1.16,149.98,0.02,3.80,0.01,15.26,155.67,76.39,1656.67,1.80,MC
136,POLICYBZR,173.55,10.05,80115.01,5.90,5.13,375.32,0.03,-4.93,0.05,39.22,164.22,-36.51,461.75,12.40,MC
137,GVT&D,87.01,35.93,80033.51,54.74,40.35,131.89,0.46,11.79,0.01,33.42,132.99,101.10,919.85,38.33,MC
138,SUZLON,24.67,2.34,78292.48,32.52,41.44,91.67,1.68,NaN,0.05,74.67,221.65,55.53,3172.50,9.96,MC
167,PATANJALI,45.04,13.12,64246.88,15.55,12.06,53.50,0.27,11.79,0.24,18.97,43.41,9.47,1426.56,5.31,MC
196,NATIONALUM,8.08,33.28,49396.63,43.96,32.72,10.81,4.05,19.54,0.00,30.80,119.36,73.83,6112.97,2.49,MC
214,ITCHOTELS,59.11,NaN,43605.73,9.63,6.66,70.51,0.11,NaN,0.01,60.03,50.12,60.09,740.85,3.95,MC
218,KAYNES,112.61,57.79,42640.67,14.28,10.74,119.54,0.10,11.67,0.19,45.11,59.48,-16.04,378.68,9.11,MC
